In [ ]:
!pip3 install pandas
!pip3 install numpy
!pip3 install scikit-learn

Importaciones

In [ ]:
import pandas as pd
import numpy as np

Carga de los datos y filtrado

In [ ]:
wind_ava = pd.read_csv('wind_ava.csv.gz', compression="gzip")
wind_ava = wind_ava.filter(regex='^(datetime|energy|.*\.13)$')

# Apartado 1
EDA o Análisis Exploratorio de los Datos

In [ ]:
# Número de instancias y características
print(f'Número de instancias: {wind_ava.shape[0]}')
print(f'Número de características: {wind_ava.shape[1]}')

# Variables categóricas y numéricas
categorical_vars = wind_ava.select_dtypes(include=['object']).columns
numerical_vars = wind_ava.select_dtypes(include=['int64', 'float64']).columns
print(f'Variables categóricas: {categorical_vars}')
print(f'Variables numéricas: {numerical_vars}')

# Valores faltantes
zero_count = (wind_ava == 0).sum().sum()
print(f"Número de valores que son 0: {zero_count}")

# Columnas constantes
constant_columns = [col for col in wind_ava.columns if wind_ava[col].nunique() <= 1]
print(f'Columnas constantes: {constant_columns}')

# Determinar si es un problema de regresión o clasificación
if wind_ava['energy'].nunique() > 2:
    print('Es un problema de regresión')
else:
    print('Es un problema de clasificación')

# Apartado 3

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.impute import SimpleImputer


# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(wind_ava[numerical_vars], wind_ava['energy'], test_size=0.3, random_state=100472166)

# Definir los métodos de escalado para evaluar
scalers = {
    'StandardScaler': StandardScaler(),
    'MinMaxScaler': MinMaxScaler(),
    'RobustScaler': RobustScaler()
}

# Crear un imputador para reemplazar los valores 0 con la media
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)

# Evaluar KNN con diferentes métodos de escalado
results = {}
for scaler_name, scaler in scalers.items():
    # Imputar los valores 0 en los datos de entrenamiento
    X_train_imputed = imputer.fit_transform(X_train)
    
    # Escalar los datos de entrenamiento
    X_train_scaled = scaler.fit_transform(X_train_imputed)
    
    # Entrenar el modelo KNN
    knn = KNeighborsRegressor()
    knn.fit(X_train_scaled, y_train)
    
    # Imputar los valores 0 en los datos de prueba
    X_test_imputed = imputer.transform(X_test)
    
    # Escalar los datos de prueba
    X_test_scaled = scaler.transform(X_test_imputed)
    
    # Evaluar el modelo
    score = knn.score(X_test_scaled, y_test)
    
    # Almacenar los resultados
    results[scaler_name] = score

for scaler_name, score in results.items():
    print(f"Variable Name: {scaler_name}")
    print(f"Variable Score: {score}")
    print()

# Elegir el método de escalado con el mejor rendimiento
mejor_scaler = max(results, key=results.get)
print(f"El mejor método de escalado es {mejor_scaler} con un puntaje de {results[mejor_scaler]}")

# Apartado 4

In [ ]:
import time
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.svm import SVR
from sklearn.dummy import DummyRegressor

# Crear instancias de los modelos
knn = KNeighborsRegressor()
tree = DecisionTreeRegressor()
linear = LinearRegression()
lasso = Lasso()
svm = SVR()
dummy = DummyRegressor(strategy='mean')

# Crear listas para almacenar los tiempos de entrenamiento y los errores
times = []
errors = []

# Entrenar y evaluar los modelos
models = [knn, tree, linear, lasso, svm, dummy]
model_names = ['KNN', 'Árboles de Regresión', 'Regresión Lineal', 'Regresión Lasso', 'SVM', 'Dummy']

for model, name in zip(models, model_names):
    start_time = time.time()
    model.fit(X_train_scaled, y_train)
    end_time = time.time()
    train_time = end_time - start_time
    times.append(train_time)
    
    y_pred = model.predict(X_test_scaled)
    error = mean_squared_error(y_test, y_pred)
    errors.append(error)
    
    print(f"Modelo: {name}")
    print(f"Tiempo de entrenamiento: {train_time} segundos")
    print(f"Error cuadrático medio: {error}")
    print("------------------------")

In [ ]:
from sklearn.model_selection import GridSearchCV

# Hiperparámetros
knn_params = {'n_neighbors': [3, 5, 7]}
tree_params = {'max_depth': [None, 5, 10]}
linear_params = {'fit_intercept': [True, False]}
lasso_params = {'alpha': [0.1, 0.5, 1.0]}
svm_params = {'C': [1.0, 10.0, 100.0]}

# Realizamos GridSearchCV para cada modelo
knn_grid = GridSearchCV(KNeighborsRegressor(), knn_params)
tree_grid = GridSearchCV(DecisionTreeRegressor(), tree_params)
linear_grid = GridSearchCV(LinearRegression(), linear_params)
lasso_grid = GridSearchCV(Lasso(), lasso_params)
svm_grid = GridSearchCV(SVR(), svm_params)

# Ajusta los modelos con los mejores hiperparámetros
knn_grid.fit(X_train_scaled, y_train)
tree_grid.fit(X_train_scaled, y_train)
linear_grid.fit(X_train_scaled, y_train)
lasso_grid.fit(X_train_scaled, y_train)
svm_grid.fit(X_train_scaled, y_train)

# Obtenemos los mejores hiperparámetros y puntuaciones para cada modelo
best_knn_params = knn_grid.best_params_
best_knn_score = knn_grid.best_score_

best_tree_params = tree_grid.best_params_
best_tree_score = tree_grid.best_score_

best_linear_params = linear_grid.best_params_
best_linear_score = linear_grid.best_score_

best_lasso_params = lasso_grid.best_params_
best_lasso_score = lasso_grid.best_score_

best_svm_params = svm_grid.best_params_
best_svm_score = svm_grid.best_score_

# Imprime los resultados
print("KNN:")
print("Mejores Hiperparámetros:", best_knn_params)
print("Mejor Puntuación:", best_knn_score)
print("------------------------")

print("Árbol de Decisión:")
print("Mejores Hiperparámetros:", best_tree_params)
print("Mejor Puntuación:", best_tree_score)
print("------------------------")

print("Regresión Lineal:")
print("Mejores Hiperparámetros:", best_linear_params)
print("Mejor Puntuación:", best_linear_score)
print("------------------------")

print("Lasso:")
print("Mejores Hiperparámetros:", best_lasso_params)
print("Mejor Puntuación:", best_lasso_score)
print("------------------------")

print("SVM:")
print("Mejores Hiperparámetros:", best_svm_params)
print("Mejor Puntuación:", best_svm_score)
print("------------------------")

Basándonos en los resultados obtenidos, llegamos a las siguientes conclusiones:

En términos de puntuación, la Regresión Lineal obtuvo un 1.0, lo que indica un muy buen ajuste a los datos. En cuanto a factores como la interpretación de los resultados y el rendimiento computacional la Regresión Lineal también mostraba muy buenos resultados, por lo que consideramos que es el mejor método.

En cuanto a velocidad, el método más rápido es el "Dummy" con 0.0 segundos (es decir, es un método prácticamente instantáneo para este conjunto de datos). Sin embargo, los resultados del resto de métodos son obviamente mejores que los obtenidos con un "DummyRegressor". Si bien es cierto que este tipo de regresor nos permite ahorrar tiempo, el error cuadrático medio es mucho más alto que el de los demás.

En general, existe un equilibrio entre el tiempo de ejecución y la mejora de resultados. Por ejemplo, los árboles de regresión tardan más que el método KNN, pero al mismo tiempo KNN muestra peores resultados que los árboles de regresióm. Esto en nuestro análisis de los datos no se cumple siempre de todos modos, ya que el método de regresión lineal es más rápido que varios de los demás y aun así presenta el mejor resultado.

El ajuste de hiperparámetros también ayuda a mejorar el rendimiento de los modelos en comparación con los valores por defecto. En este caso, hemos utilizado GridSearchCV para buscar los mejores hiperparámetros para cada modelo. Por ejemplo, para el método SMV y para la regresión Lasso se consideran como los mejores hiperparámetros el 100.0 y el 0.1, respectivamente. En cambio, el resto de métodos (regresión lineal, KNN y árboles de regresión).

Por último, podríamos extraer atributos relevantes dependiendo del modelo utilizado, ya que algunos (como los árboles de decisión) proporcionan información sobre la importancia de los atributos. Por ejemplo, se puede obtener la importancia de los atributos en uno de estos árboles mediante el atributo feature_importances_.

# Apartado 5